In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor
import warnings
warnings.filterwarnings('ignore')

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [8]:
def PCA_test(matrix, pca):

    reducedMatrixPCA = pca.transform(matrix)

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return reducedDf

In [9]:
def inverse_pca(matrix,pca):
    m = matrix.copy()
    return pca.inverse_transform(m)

In [10]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [11]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [12]:
def pca_performance(trainmatrix,testmatrix, components):
    rmseList = []
    r2List = []
    for n in components:
        scaler, s_train_matrix = standardize(trainmatrix)
        s_test_matrix = scaler.transform(testmatrix)

        pca,pcaTrain = getPCAFeatures(s_train_matrix,n=n)
        pcaTest = PCA_test(s_test_matrix, pca)
        
        network_prediction = inverse_pca(pcaTest,pca)
        network_prediction = inverse_standardize(network_prediction, scaler)

        r2Score = r2_score(testmatrix, network_prediction, multioutput='variance_weighted')
                
        r2List.append(r2Score)
    
    return r2List

In [13]:
def nonlinearperformance(trainmatrix,testmatrix,components, maxlag=12):
    r2List = []
    for n in components:
        print(n)
        scaler, s_train_matrix = standardize(trainmatrix)
        s_test_matrix = scaler.transform(testmatrix)

        pca,pcaTrain = getPCAFeatures(s_train_matrix,n=n)
        pcaTest = PCA_test(s_test_matrix, pca)

#         maxlag = 12
        DateColumns = ['Date', 'Hour']
        lagColumns = [c for c in pcaTrain.columns if c not in DateColumns]

        dataset_train = addLag(pcaTrain, maxlag)

        dataset_test = addLag(pcaTest, maxlag)

        X_train = dataset_train.drop(lagColumns , axis = 1)
        X_test = dataset_test.drop(lagColumns , axis = 1)
        y_train = dataset_train[lagColumns]
        y_test = dataset_test[lagColumns]
#         print(X_train.shape)
#         print(X_test.shape)
#         print(y_train.shape)
#         print(y_test.shape)

        rf2 = RandomForestRegressor(random_state = 0, n_estimators=200, 
                                   min_samples_split=10,
                                   min_samples_leaf= 3, 
                                   max_features= 'sqrt',
                                   max_depth= 30, 
                                   bootstrap= True)

        rf2.fit(X_train,y_train)

        pca_prediction = rf2.predict(X_test)

        network_prediction = inverse_pca(pca_prediction,pca)

        network_prediction = inverse_standardize(network_prediction, scaler)

        r2Score = r2_score(testmatrix[maxlag:], network_prediction, \
                           multioutput='variance_weighted')
        
        r2List.append(r2Score)
    return r2List

#### Preparing Data

In [14]:
hub = 'LGA'
tune_hyp_params = False
pca_comps = 6

In [15]:
dataDir = '/home/mingyi/Dropbox/UrbanTemporalNetworks/processedData/'
file = dataDir + hub + 'VehicleByHour.csv'

In [16]:
# file = '/home/urwa/Documents/Projects/NYU Remote/project/data/JfkVehiceByHour.csv'

In [17]:
data = loadData(file)

Raw shape:  (2251320, 4)
Days:  365


In [18]:
data = getTimeSeries(data)

In [19]:
data.shape

(8760, 257)

In [20]:
data

DOLocationID     1    2    3    4    5    6    7    8    9    10   ...  254  \
Date       Hour                                                    ...        
2018-01-01 0       0    0    0    0    0    0    1    0    1    0  ...    0   
           1       0    0    0    0    0    0    2    0    0    0  ...    0   
           2       0    0    0    0    0    0    0    0    0    0  ...    0   
           3       0    0    0    0    0    0    0    0    0    0  ...    0   
           4       1    0    0    0    0    0    0    0    0    0  ...    0   
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2018-12-31 19      0    0    1    0    0    0    8    0    1    0  ...    0   
           20      0    0    0    2    0    0   12    0    2    0  ...    1   
           21      0    0    1    1    0    0   18    0    1    0  ...    1   
           22      0    0    0    0    0    0   14    1    1    1  ...    1   
           23      1    0    1    2    0    0    7    0    1    1  ...    2   

DOLocationID     255  256  257  258  259  260  261  262  263  
Date       Hour                                               
2018-01-01 0       0    1    0    0    2    1    0    4    1  
           1       0    0    0    0    0    0    0    0    1  
           2       0    0    0    0    0    0    0    0    0  
           3       0    0    0    0    0    0    0    0    0  
           4       0    0    0    0    0    0    0    0    0  
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  
2018-12-31 19      9    5    3    0    1    4    1    5    5  
           20      7    4    1    3    1    6    2   10   16  
           21      1    3    1    0    0    5    1    6    8  
           22      4    0    2    0    0    2    0    7    5  
           23      6    4    0    2    2    3    1    8    9  

[8760 rows x 257 columns]

In [21]:
matrix = data.values.astype(np.float64)

In [22]:
scaler, s_matrix = standardize(matrix)

In [23]:
pca,pcaData = getPCAFeatures(s_matrix,n=pca_comps)

In [24]:
pcaData.index = data.index
pcaData = pcaData.reset_index()

In [25]:
pcaData.to_csv('../../processedData/%spca%s.csv'%(hub.upper(),pca_comps),
               index=False)

In [25]:
externalDataDir = "/home/mingyi/Dropbox/UrbanTemporalNetworks/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [26]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,3,89,67,156,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,18/1/1 1:00,0,17,8,25,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [27]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,3,89,67,156,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,0,17,8,25,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [28]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [29]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [30]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [31]:
extDf = extDf[selected_columns]

In [32]:
print(pcaData.shape)
print(extDf.shape)

(8760, 8)
(8760, 14)


In [33]:
pcaData['Date'] = pd.to_datetime(pcaData['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [34]:
pcaData = pd.merge(pcaData,extDf, on=['Date', 'Hour'], how='inner')
print(pcaData.shape)

(8760, 20)


In [35]:
pcaData.columns

Index(['Date', 'Hour', '1', '2', '3', '4', '5', '6', 'Dow', 'arrival',
       'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd',
       'participation', 'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object')

In [36]:
lagColumns = ['1', '2', '3', '4', '5', '6', 'arrival']
# lagColumns = ['1', '2', '3', 'arrival']

DateColumns = ['Date']

targetColumns = ['1', '2', '3', '4', '5', '6']

In [37]:
maxlag = 12

pcaData_lag = addLag(pcaData, maxlag, lagColumns)

pcaData_lag.shape

(8748, 104)

In [38]:
data.columns

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            254, 255, 256, 257, 258, 259, 260, 261, 262, 263],
           dtype='int64', name='DOLocationID', length=257)

In [39]:
pcaData_lag

,Date,Hour,1,2,3,4,5,6,Dow,arrival,...,5_lag_11,6_lag_11,arrival_lag_11,1_lag_12,2_lag_12,3_lag_12,4_lag_12,5_lag_12,6_lag_12,arrival_lag_12
12,2018-01-01,12,0.586102,-2.609485,0.210539,0.191380,0.653304,1.521694,0,14,...,-0.022617,-0.037017,0.0,-7.249291,-2.710130,1.318519,0.197295,1.209991,0.525303,3.0
13,2018-01-01,13,-0.392435,-2.865557,-1.814105,0.687439,-0.327043,-0.135517,0,20,...,-0.208235,0.171843,1.0,-11.299151,-1.028730,-0.578698,0.029822,-0.022617,-0.037017,0.0
14,2018-01-01,14,3.885274,-3.450488,-2.718940,-0.159802,-0.781531,-0.635154,0,25,...,-0.110618,0.000919,0.0,-11.798621,-0.934869,-0.914789,0.420583,-0.208235,0.171843,1.0
15,2018-01-01,15,6.615891,-3.833182,-3.267279,0.924116,-2.405509,0.231102,0,20,...,-0.035521,-0.233022,0.0,-11.828333,-0.896374,-0.895143,0.337762,-0.110618,0.000919,0.0
16,2018-01-01,16,4.607887,-3.462946,-5.374247,-0.258785,-1.512452,-0.046257,0,24,...,-0.093348,-0.154143,0.0,-11.576102,-0.854354,-0.806231,-0.141248,-0.035521,-0.233022,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-12-31,19,0.181023,-5.072704,1.453587,0.869776,-0.788511,0.801764,0,11,...,-1.045396,0.414158,22.0,-8.746955,-1.664700,0.628280,1.041308,0.211545,1.041251,11.0
8756,2018-12-31,20,3.237670,-5.588828,1.612167,0.561244,-1.319047,1.059549,0,13,...,-2.992436,-0.475864,23.0,-1.209359,-2.811690,0.627818,0.354293,-1.045396,0.414158,22.0
8757,2018-12-31,21,-1.468272,-4.490884,1.753691,1.318519,0.385046,-1.502815,0,8,...,-1.816930,1.000102,28.0,3.261662,-3.351066,3.932088,1.690045,-2.992436,-0.475864,23.0
8758,2018-12-31,22,-0.010294,-5.367257,3.613050,1.387191,0.487512,1.198915,0,8,...,-2.016839,2.188138,11.0,9.645854,-5.972764,1.901887,2.463275,-1.816930,1.000102,28.0


In [46]:
CommR2List = []
EdgeR2List = []
residualDf_list = []
rawList = []
networkPrediction = pd.DataFrame()

for m in range(1,13):
    print()

    print("month: ",m)
    month_index  = pd.to_datetime(pcaData_lag.Date).dt.month == m

    dataset_train = pcaData_lag[~month_index]
    dataset_test = pcaData_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)


    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]



    rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                               min_samples_split=3,
                               min_samples_leaf= 2, 
                               max_features= 'sqrt',
                               max_depth= None, 
                               bootstrap= False)

    rf2.fit(X_train,y_train)

    print("Train R2: ",rf2.score(X_train,y_train))
    test_r2 = rf2.score(X_test,y_test)
    print("Test R2: ",test_r2)


    pca_prediction = rf2.predict(X_test)

    residual = y_test - pca_prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)

    network_prediction = inverse_pca(pca_prediction,pca)

    network_prediction = inverse_standardize(network_prediction, scaler)
    
    # relu to convert all prediction to positive
#     network_prediction = np.log(1+np.e**network_prediction)
    # round up negative values to 0
#     network_prediction = np.where(network_prediction<0,0,network_prediction)
    
    network_prediction_df = pd.DataFrame(network_prediction)
    network_prediction_df.columns = data.columns
    networkPrediction = pd.concat([networkPrediction,network_prediction_df])
    edgeMonthIndex = [False] * maxlag + list(month_index)
    edge_r2 = r2_score(data[edgeMonthIndex], network_prediction, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)


    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)
#     rawList.append()


month:  1
Train Size:  (8016, 104)
Test Size:  (732, 104)
Train R2:  0.9909043423360494
Test R2:  0.8372213423860555
Edge R2:  0.7520046653534043

month:  2
Train Size:  (8076, 104)
Test Size:  (672, 104)
Train R2:  0.9907635814362779
Test R2:  0.8796086792864446
Edge R2:  0.778273809381598

month:  3
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9906807899323753
Test R2:  0.896102152223058
Edge R2:  0.7856054610189863

month:  4
Train Size:  (8028, 104)
Test Size:  (720, 104)
Train R2:  0.9908508715777669
Test R2:  0.8726912181009698
Edge R2:  0.7945829627799328

month:  5
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9906287456350282
Test R2:  0.8894082643022624
Edge R2:  0.8032717192652908

month:  6
Train Size:  (8028, 104)
Test Size:  (720, 104)
Train R2:  0.9906812299682184
Test R2:  0.8889561701918482
Edge R2:  0.7914320542838518

month:  7
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9907656776855535
Test R2:  0.8682546675893456
Ed

In [47]:
networkPrediction['Date'] = data.reset_index().iloc[12:]['Date'].values
networkPrediction['Hour'] = data.reset_index().iloc[12:]['Hour'].values
networkPrediction.to_csv('/home/mingyi/Dropbox/UrbanTemporalNetworks/prediction/%sPCA%s.csv'%(hub,pca_comps),index=False)

In [45]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.8737981183247389
0.7757471879414016


In [48]:
res_df = pd.concat(residualDf_list, axis = 0)
res_df.to_csv('../../Resid/'+hub+'PCA'+str(pca_comps)+'RFCV.csv',index=False)